In [31]:
import geopandas as gpd
import scipy as sp
import numpy as np
from shapely.geometry.polygon import Polygon
from libpysal.graph import Graph

In [2]:
gdf = gpd.read_parquet('/data/uscuni-ulce/cadastre_buildings_standardised/buildings_czechia.pq')

In [6]:
centroids = gdf.geometry.representative_point()

In [7]:
from scipy.spatial import Delaunay

In [18]:
points = np.vstack((centroids.x, centroids.y)).T

In [19]:
%%time
tri = Delaunay(points)

CPU times: user 17.9 s, sys: 706 ms, total: 18.6 s
Wall time: 18.6 s


In [20]:
%%time
coord_groups = [tri.points[x] for x in tri.simplices]
polygons = gpd.GeoSeries([Polygon(x) for x in coord_groups], name='geometry', crs=gdf.crs)

CPU times: user 1min 4s, sys: 943 ms, total: 1min 5s
Wall time: 1min 5s


In [48]:
polygons.area.describe().iloc[1:-1]

mean     10797.864732
std     675893.150976
min          0.004450
25%        185.255010
50%        446.261989
75%       1156.523512
dtype: float64

In [49]:
urban_triangles = polygons[polygons.area <= polygons.area.mean()]

In [33]:
%%time
graph = Graph.build_fuzzy_contiguity(polygons, buffer=0.1)

CPU times: user 9min 33s, sys: 10.3 s, total: 9min 43s
Wall time: 9min 43s


In [50]:
%%time
subgraph = graph.subgraph(urban_triangles.index.values)

CPU times: user 2min 16s, sys: 4.96 s, total: 2min 21s
Wall time: 2min 20s


In [51]:
%%time
dissolved = urban_triangles.to_frame().dissolve(subgraph.component_labels)

CPU times: user 2min 8s, sys: 237 ms, total: 2min 9s
Wall time: 2min 9s


In [52]:
dissolved['area'] = dissolved.area

In [55]:
# dissolved.sort_values('area', ascending=False).iloc[:1000].explore()